[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/supply_chain/02_Supply_Chain_Risk_Management.ipynb)

# Supply Chain Risk Management - Dependency Analysis & Reasoning

## Overview

This notebook demonstrates **supply chain risk management** using Semantica with focus on **dependency analysis**, **risk pattern detection**, and **external feed correlation**. The pipeline detects risks in the supply chain by analyzing dependencies and external feeds using reasoning and analytics.

### Key Features

- **Dependency Analysis**: Analyzes supply chain dependencies using graph reasoning
- **Risk Pattern Detection**: Detects risk patterns in the supply chain
- **External Feed Correlation**: Correlates external feeds with supply chain data
- **Reasoning**: Emphasizes reasoning and analytics for risk detection
- **Risk Mitigation**: Identifies mitigation strategies through graph analysis

### Pipeline Architecture

1. **Phase 0**: Setup & Configuration
2. **Phase 1**: Supply Chain & External Feed Ingestion
3. **Phase 2**: Entity Extraction (Dependency, Risk, Disruption, Impact, Mitigation)
4. **Phase 3**: Supply Chain Knowledge Graph Construction
5. **Phase 4**: Dependency Analysis
6. **Phase 5**: Risk Pattern Detection
7. **Phase 6**: External Feed Correlation
8. **Phase 7**: Visualization & Risk Reporting

---

## Installation


In [ ]:
%pip install -qU semantica networkx matplotlib plotly pandas groq


---

## Phase 0: Setup & Configuration


In [ ]:
import os
from semantica.core import Semantica, ConfigManager
from semantica.reasoning import GraphReasoner

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "your-key")

config_dict = {
    "project_name": "Supply_Chain_Risk_Management",
    "extraction": {"provider": "groq", "model": "llama-3.1-8b-instant"},
    "knowledge_graph": {"backend": "networkx"}
}

config = ConfigManager().load_from_dict(config_dict)
core = Semantica(config=config)
print("Configured for supply chain risk management with reasoning focus")


---

## Phase 1: Real Data Ingestion (Supply Chain RSS Feeds)

Ingest supply chain risk data from RSS feeds.


In [ ]:
from semantica.ingest import FeedIngestor, FileIngestor
from semantica.normalize import TextNormalizer
from semantica.conflicts import ConflictDetector
import os

os.makedirs("data", exist_ok=True)

# Ingest from supply chain RSS feeds
supply_feeds = [
    # Add supply chain news RSS feeds here
]

documents = []
for feed_url in supply_feeds:
    try:
        feed_ingestor = FeedIngestor()
        feed_documents = feed_ingestor.ingest(feed_url, method="rss")
        documents.extend(feed_documents)
    except Exception as e:
        print(f"Feed ingestion failed: {e}")

# Fallback: Sample data
if not documents:
    risk_data = """
    Supplier A depends on raw materials from Region R1 (high risk region).
    Disruption in Region R1 impacts Supplier A, causing supply chain risk.
    External feed: Weather alert in Region R1 may disrupt logistics.
    Risk mitigation: Identify alternative suppliers in Region R2.
    """
    with open("data/supply_chain_risks.txt", "w") as f:
        f.write(risk_data)
    documents = FileIngestor().ingest("data/supply_chain_risks.txt")
    print(f"Ingested {len(documents)} documents from sample data")

# Normalize risk data
normalizer = TextNormalizer()
normalized_documents = []
for doc in documents:
    normalized_text = normalizer.normalize(
        doc.content if hasattr(doc, 'content') else str(doc),
        clean_html=True,
        normalize_entities=True,
        remove_extra_whitespace=True
    )
    normalized_documents.append(normalized_text)

print(f"Normalized {len(normalized_documents)} documents")

# Build supply chain risk knowledge graph
result = core.build_knowledge_base(
    sources=normalized_documents,
    custom_entity_types=["Dependency", "Risk", "Disruption", "Impact", "Mitigation"],
    graph=True
)

kg = result["knowledge_graph"]
entities = result["entities"]

# Detect conflicts in risk data
detector = ConflictDetector()
conflicts = detector.detect_conflicts(entities, kg.get("relationships", []))

print(f"Built risk management KG with {len(kg.get('entities', []))} entities")
print(f"Detected {len(conflicts)} conflicts in risk data")
if conflicts:
    resolved = detector.resolve_conflicts(conflicts, strategy="highest_confidence")
    print(f"Resolved {len(resolved)} conflicts")
print("Focus: Dependency analysis, risk pattern detection, external feed correlation, reasoning")


In [ ]:
# Analyze dependencies using reasoning
reasoner = GraphReasoner(kg)
dependencies = reasoner.find_patterns(pattern_type="dependency")
risk_patterns = reasoner.find_patterns(pattern_type="risk")

# Identify high-risk dependencies
high_risk = [e for e in kg.get("entities", []) 
             if e.get("type") == "Risk" and 
             "high" in str(e.get("text", "")).lower()]

print(f"Dependency analysis: {len(dependencies)} dependencies identified")
print(f"Risk pattern detection: {len(risk_patterns)} risk patterns found")
print(f"High-risk items: {len(high_risk)} high-risk dependencies flagged")
print("This cookbook emphasizes reasoning and analytics for risk management")


---

## Phase 7: Visualization


In [ ]:
from semantica.visualization import KGVisualizer

visualizer = KGVisualizer()
visualizer.visualize(kg, output_path="supply_chain_risk_kg.html")

print("Supply chain risk management analysis complete")
print("Emphasizes: Dependency analysis, risk pattern detection, external feed correlation, reasoning")
